# TensorFlow MNIST II

Computer vision - build a neural network.

*The content of this notebook is based on the official TensorFlow tutorial "TensorFlow 2 quickstart for beginners". Here is the original source code:*

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

## Data preparation

In [1]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [14]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Neural network design

Build the `tf.keras.Sequential` model by stacking layers. The ``Dropout`` layer randomly sets input units to 0 with a frequency of rate 0.2 at each step during training time, which helps prevent overfitting. 

In [15]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

## Optimizer and loss function

Next, we need to choose an **optimizer** and **loss** function for training:

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [16]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.54341495, -0.21888089, -0.9366398 , -0.72691745, -0.14280474,
        -0.93294215,  0.00493893,  0.23920178, -0.5972141 , -0.02694894]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [17]:
tf.nn.softmax(predictions).numpy()

array([[0.20353417, 0.09496782, 0.04632951, 0.05713984, 0.10247453,
        0.04650114, 0.11879008, 0.1501479 , 0.06505316, 0.11506188]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [18]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [19]:
loss_fn(y_train[:1], predictions).numpy()

3.0682783

In [20]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Model training

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [21]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 823us/step - loss: 0.2971 - accuracy: 0.9135
Epoch 2/5
1875/1875 [==============================] - 2s 970us/step - loss: 0.1434 - accuracy: 0.9569
Epoch 3/5
1875/1875 [==============================] - 1s 717us/step - loss: 0.1083 - accuracy: 0.9668
Epoch 4/5
1875/1875 [==============================] - 1s 734us/step - loss: 0.0891 - accuracy: 0.9728
Epoch 5/5
1875/1875 [==============================] - 2s 815us/step - loss: 0.0780 - accuracy: 0.9755


## Model evaluation

The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [22]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0814 - accuracy: 0.9760 - 215ms/epoch - 687us/step


[0.08140749484300613, 0.9760000109672546]

The image classifier is now trained to ~98% accuracy on this dataset.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [23]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [24]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[5.40054032e-07, 5.28068655e-09, 8.55695725e-06, 9.49303503e-05,
        9.11432713e-13, 2.27035581e-08, 1.87984258e-14, 9.99893308e-01,
        1.33198921e-06, 1.40733334e-06],
       [6.38863412e-07, 4.65077907e-03, 9.95305955e-01, 4.32945581e-06,
        1.22102849e-12, 3.83286060e-05, 3.58877585e-08, 6.27718849e-11,
        2.93874312e-08, 3.68349884e-09],
       [4.11219162e-07, 9.97538924e-01, 8.47957737e-04, 4.33883215e-05,
        2.19761168e-05, 6.66951601e-05, 3.85721214e-05, 9.11290990e-04,
        5.30206133e-04, 6.71609655e-07],
       [9.99988079e-01, 1.04602806e-10, 8.74882062e-06, 4.63699443e-08,
        3.74681761e-08, 8.96031167e-07, 8.58595001e-07, 8.02475938e-07,
        2.39282318e-08, 4.66454878e-07],
       [1.24081153e-05, 2.69615330e-10, 7.30154397e-06, 5.43351121e-07,
        9.96388555e-01, 1.11955351e-05, 2.35643552e-06, 7.20459328e-04,
        4.44048555e-06, 2.85285013e-03]], dtype=float32)>